In [ ]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from operator import attrgetter

In [ ]:
df = pd.read_csv('/Users/steve/Documents/DSB_M2/bcg/transactions_dataset.csv', sep = ';')

In [ ]:
df.order_channel.value_counts()

In [ ]:
missing_values = df.isnull().sum()
print(missing_values)

In [ ]:

df['date_order'] = pd.to_datetime(df['date_order'])
df['date_invoice'] = pd.to_datetime(df['date_invoice'])

df['order_invoice_delta'] = (df['date_invoice'] - df['date_order']).dt.days

print(df[['date_order', 'date_invoice', 'order_invoice_delta']].head())


In [ ]:


plt.figure(figsize=(10, 6))
plt.hist(df['order_invoice_delta'], bins=3, color='skyblue', edgecolor='black')
plt.title('Distribution of Order-Invoice Delta')
plt.xlabel('Order-Invoice Delta (days)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
df.order_invoice_delta.describe()

In [ ]:
df.order_invoice_delta.value_counts()

In [ ]:
df.sales_net.describe()

In [ ]:
mean_net_spend_per_channel = df.groupby('order_channel')['sales_net'].mean()
print(mean_net_spend_per_channel)


In [ ]:
df[df["order_invoice_delta"]>0]

In [ ]:
df["client_id"].value_counts()

In [ ]:
net_spend_per_product = df.groupby('product_id')['sales_net'].sum().sort_values(ascending=False)
print(net_spend_per_product)

In [ ]:
net_spend_per_product_top_n = net_spend_per_product.head(20)

plt.figure(figsize=(12, 8))
net_spend_per_product_top_n.plot(kind='bar')
plt.title('Net Spend per Product (Top 10)')
plt.xlabel('Product ID')
plt.ylabel('Total Net Spend')
plt.show()

In [ ]:
net_spend_per_store = df.groupby('branch_id')['sales_net'].sum().sort_values(ascending=False)
print(net_spend_per_store)

In [ ]:
net_spend_per_store_top_n = net_spend_per_store.head(20)

plt.figure(figsize=(12, 8))
net_spend_per_store_top_n.plot(kind='bar')
plt.title('Net Spend per Store (Top 10)')
plt.xlabel('Product ID')
plt.ylabel('Total Net Spend')
plt.show()

In [ ]:
top_10_clients_by_sales_net = df.groupby('client_id')['sales_net'].sum().sort_values(ascending=False).head(10)
print(top_10_clients_by_sales_net)


In [ ]:
top_10_clients_ids = top_10_clients_by_sales_net.index
top_clients_df = df[df['client_id'].isin(top_10_clients_ids)].head(10)


plt.figure(figsize=(12, 8))
top_clients_df['branch_id'].value_counts().plot(kind='bar')
plt.title('Frequency of Branch IDs for Top 100 Clients by Net Sales')
plt.xlabel('Branch ID')
plt.ylabel('Frequency')
plt.show()

In [ ]:
grouped_orders = df.groupby(['client_id', 'product_id']).agg({'date_order': ['count', 'min', 'max'], 'quantity': 'sum'}).reset_index()
grouped_orders.columns = ['client_id', 'product_id', 'total_orders', 'first_order_date', 'last_order_date', 'total_quantity']


In [ ]:
repeated_orders = grouped_orders[grouped_orders['total_orders'] > 1]

repeated_orders['average_order_frequency'] = (repeated_orders['last_order_date'] - repeated_orders['first_order_date']) / (repeated_orders['total_orders'] - 1)


In [ ]:
repeated_orders.sort_values(by = 'total_orders', ascending=False).head(40)

In [ ]:

subscription_like = repeated_orders[(repeated_orders['average_order_frequency'] > pd.Timedelta(days=25)) & (repeated_orders['average_order_frequency'] < pd.Timedelta(days=35))]
client_id = subscription_like.iloc[50]['client_id'] # Plotting order frequency for a specific client
client_orders = df[df['client_id'] == client_id]

plt.figure(figsize=(10, 6))
plt.plot(client_orders['date_order'], client_orders['quantity'], marker='o')
plt.title(f'Order Frequency for Client {client_id}')
plt.xlabel('Date')
plt.ylabel('Quantity')
plt.show()


In [ ]:
volume_categories = df.groupby('client_id')['quantity'].sum().reset_index(name='TotalQuantity')
volume_categories.sort_values(by='TotalQuantity', ascending=False).head(35)

In [ ]:
day_total_quantity = df.groupby('date_order')['quantity'].sum().reset_index(name='TotalQuantity')
day_total_quantity['date_order'] = pd.to_datetime(day_total_quantity['date_order'])
day_total_quantity.set_index('date_order', inplace=True)

day_total_quantity['day_of_week'] = day_total_quantity.index.day_name()
plt.figure(figsize=(15, 7))
sns.barplot(x='day_of_week', y='TotalQuantity', data=day_total_quantity, color = '#28BA74')
plt.title('Day of the Week Analysis')
plt.xlabel('Day of Week')
plt.ylabel('Total Quantity')
plt.show()

In [ ]:
df['day_of_week'] = df['date_order'].dt.day_name()
weekend_orders = df[df['day_of_week'].isin(['Saturday', 'Sunday'])]
weekend_order_channel_frequency = weekend_orders['order_channel'].value_counts()

plt.figure(figsize=(10, 6))
weekend_order_channel_frequency.plot(kind='bar', color = '#28BA74')
plt.title('Order Channel Frequency on Weekends')
plt.xlabel('Order Channel')
plt.ylabel('Frequency')
plt.show()


In [ ]:
df['date_order'] = pd.to_datetime(df['date_order'])
df['OrderMonth'] = df['date_order'].dt.to_period('M')
df['CohortMonth'] = df.groupby('client_id')['date_order'].transform('min').dt.to_period('M')
cohort_data = df.groupby(['CohortMonth', 'OrderMonth']).agg(n_customers=('client_id', 'nunique')).reset_index()
cohort_data['CohortIndex'] = (cohort_data.OrderMonth - cohort_data.CohortMonth).apply(attrgetter('n'))
cohort_counts = cohort_data.pivot_table(index='CohortMonth', columns='CohortIndex', values='n_customers')
cohort_sizes = cohort_counts.iloc[:,0]
retention = cohort_counts.divide(cohort_sizes, axis=0)

# Plotting the retention rates
plt.figure(figsize=(12, 8))
sns.heatmap(retention, annot=True, fmt='.0%', cmap='BuGn')
plt.title('Cohorts: User Retention')
plt.ylabel('Cohort Month')
plt.xlabel('Months after First Purchase')
plt.show()


# Modelling

In [ ]:
from BCG_Data_Strategy.src.dataloader import final_preprocessing
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
data_path = Path('/Users/steve/Documents/DSB_M2/bcg/transactions_dataset.csv')
drop_time = 180
churn_time = 180
recent_to = 180
day_diff_to = 4
total_days_to = 3

train_df = final_preprocessing(data_path, drop_time, churn_time, recent_to, day_diff_to, total_days_to)


In [ ]:
test_drop_time = 0

test_df = final_preprocessing(data_path, test_drop_time, churn_time, recent_to, day_diff_to, total_days_to)


In [ ]:
X_train = train_df.drop('churn', axis=1)
y_train = train_df['churn']

X_test = test_df.drop('churn', axis=1)
y_test = test_df['churn']

## Baseline model

In [ ]:
model = xgb.XGBClassifier(objective='binary:logistic', seed=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy}")


In [ ]:
conf_matrix = confusion_matrix(y_test, predictions)
sns.heatmap(conf_matrix, annot=True, fmt='g')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
y_pred_proba = model.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {roc_auc}")


plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


## Random search 

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'colsample_bytree': [0.3, 0.4, 0.5, 0.7],
    'min_child_weight': [1, 2, 3, 4]
}

xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', seed=42)

random_search = RandomizedSearchCV(xgb_classifier, param_distributions=param_grid, 
                                   n_iter=50, scoring='accuracy', n_jobs=-1, cv=5, 
                                   random_state=42)

random_search.fit(X_train, y_train)

best_estimator = random_search.best_estimator_
best_params = random_search.best_params_
print("Best Parameters:", best_params)

predictions = best_estimator.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy with Best Parameters: {accuracy}")


In [ ]:
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy}")

In [ ]:
conf_matrix = confusion_matrix(y_test, predictions)
sns.heatmap(conf_matrix, annot=True, fmt='g')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
y_pred_proba = best_estimator.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {roc_auc}")


plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


# Addition of more data

In [ ]:
relationship_df = pd.read_excel('/Users/steve/Documents/DSB_M2/bcg/sales_client_relationship_dataset.xlsx')

relationship_encoded = pd.get_dummies(relationship_df, columns=['quali_relation'])

train_df1 = train_df.merge(relationship_encoded, on='client_id', how='left')
test_df1 = test_df.merge(relationship_encoded, on='client_id', how='left')


In [ ]:
X_train = train_df1.drop('churn', axis=1)
y_train = train_df1['churn']

X_test = test_df1.drop('churn', axis=1)
y_test = test_df1['churn']

## Baseline model with more data
model = xgb.XGBClassifier(objective='binary:logistic', seed=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy}")


y_pred_proba = model.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {roc_auc}")


plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


# Grid Search

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'colsample_bytree': [0.3, 0.4, 0.5, 0.7],
    'min_child_weight': [1, 2, 3, 4]
}

xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', seed=42)

grid_search = GridSearchCV(xgb_classifier, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=5)

grid_search.fit(X_train, y_train)

best_estimator = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

predictions = best_estimator.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy with Best Parameters: {accuracy}")


In [ ]:
conf_matrix = confusion_matrix(y_test, predictions)
sns.heatmap(conf_matrix, annot=True, fmt='g')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
y_pred_proba = best_estimator.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {roc_auc}")


plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Loading the model with best parameters

Best Parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 200}

In [ ]:
model = xgb.XGBClassifier(colsample_bytree=0.7, 
                          learning_rate=0.1, 
                          max_depth=4, 
                          min_child_weight=4, 
                          n_estimators=200)


model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

In [ ]:
feature_importances = model.feature_importances_

# To visualize the feature importances
plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importances)), feature_importances, color = '#28BA74')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.xticks(ticks=range(len(feature_importances)), labels=X_train.columns, rotation=90)
plt.title('Feature Importance')
plt.show()


# Estimating potential savings

In [ ]:
test_df1['predicted_churn'] = model.predict(X_test)


test_df1['potential_loss'] = test_df1['client_avg_sales'] * test_df1['client_total_days']

# Filter to customers who were both predicted and actually churned
correct_predictions = test_df1[(test_df1['predicted_churn'] == 1) & (test_df1['churn'] == 1)]

# Calculate total potential revenue loss for correctly predicted churned customers
total_potential_loss = correct_predictions['potential_loss'].sum()

retention_strategy_success_rate = 0.1 
potential_savings = total_potential_loss * retention_strategy_success_rate

print(f"Total Potential Revenue Loss for Correctly Predicted Churned Customers: ${total_potential_loss}")
print(f"Potential Savings with Retention Strategies: ${potential_savings}")


In [ ]:
plt.hist(train_df['client_avg_day_diff'], bins=50, color = '#28BA74')
plt.xlabel('Average Day Difference')
plt.ylabel('Frequency')
plt.title('Histogram of Client Average Day Difference')
plt.show()
